In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import ipywidgets as widgets
from IPython.display import display, HTML, Image
import os

merged_df_path = 'merged_skillsets_cv.xlsx'  # Adjusted for direct access in the repository
merged_df = pd.read_excel(merged_df_path)

def normalize_name(name):
    """Normalize names for more flexible matching."""
    return name.lower().replace(' ', '_')

def find_person_image(name):
    """Finds the image file for a given person's name based on partial matching."""
    normalized_name = normalize_name(name)
    for file in os.listdir('.'):  # Changed to current directory
        normalized_file_name = normalize_name(file.split('.')[0])
        if normalized_name.startswith(normalized_file_name) or normalized_file_name.startswith(normalized_name):
            return file  # Return the filename directly
    return None

def extract_keywords(df, column, n_keywords=50):
    """Extracts keywords for a column using TF-IDF."""
    vectorizer = TfidfVectorizer(stop_words='english', max_features=n_keywords)
    corpus = df[column].dropna().astype(str).tolist()
    X = vectorizer.fit_transform(corpus)
    return sorted(vectorizer.get_feature_names_out())

search_categories = ['Grade', 'Skills', 'Qualifications', 'Projects']
dropdowns = {
    'Grade': widgets.Dropdown(
        options=['Select'] + sorted(merged_df['Grade'].dropna().unique().tolist()),
        description='Grade'
    )
}

for category in ['Skills', 'Qualifications', 'Projects']:
    dropdowns[category] = widgets.Dropdown(
        options=['Select'] + extract_keywords(merged_df, category),
        description=category
    )

names_output = widgets.Output()
details_output = widgets.Output()

def filter_df(selected_filters):
    """Filters the DataFrame based on selected dropdown options."""
    filtered_df = merged_df
    for category, dropdown in dropdowns.items():
        if dropdown.value and dropdown.value != 'Select':
            column_name = category
            filtered_df = filtered_df[filtered_df[column_name].str.contains(dropdown.value, case=False, na=False)]
    return filtered_df

def display_names(selected_filters=None):
    """Displays names as buttons based on the current selection."""
    if selected_filters is None:
        selected_filters = {cat: dd.value for cat, dd in dropdowns.items()}
    filtered_df = filter_df(selected_filters)
    with names_output:
        names_output.clear_output()
        if not filtered_df.empty:
            for name in filtered_df['Name'].dropna().unique():
                btn = widgets.Button(description=name, button_style='primary')
                btn.on_click(lambda b, name=name: display_person_details(name))
                display(btn)

def display_person_details(name):
    """Displays details and picture for a selected name."""
    person_data = merged_df[merged_df['Name'].str.contains(name, case=False, na=False)].iloc[0]
    image_path = find_person_image(name)
    
    details_html = f"<h3>Details for {name}</h3>" + \
                   f"<p><b>Position:</b> {person_data.get('Position', 'N/A')}</p>" + \
                   f"<p><b>Education:</b> {person_data.get('Education', 'N/A')}</p>" + \
                   f"<p><b>Areas of Expertise:</b> {person_data.get('Areas of Expertise', 'N/A')}</p>" + \
                   f"<p><b>Relevant Projects:</b> {person_data.get('Relevant Projects', 'N/A')}</p>" + \
                   f"<p><b>CV:</b> <a href='{person_data.get('CV', '#')}' target='_blank'>Open CV</a></p>"
                   
    with details_output:
        details_output.clear_output()
        if image_path:
            display(Image(filename=image_path, width=200, height=200))
        display(HTML(details_html))

for category, dropdown in dropdowns.items():
    dropdown.observe(on_dropdown_change, names='value')

reset_button = widgets.Button(description="Reset", button_style='danger')
def reset_all(*args):
    """Resets all dropdowns to 'Select' and clears outputs."""
    for dropdown in dropdowns.values():
        dropdown.value = 'Select'
    names_output.clear_output()
    details_output.clear_output()
reset_button.on_click(reset_all)

display(Image(filename=r'C:\Users\melkhadra001\Downloads\Logo-pwc.png', width=200, height=100))
display(widgets.VBox([*dropdowns.values(), reset_button]))
display(names_output)
display(details_output)

for category in search_categories:
    column_name = 'PROJECTS' if category == 'Projects' else category
    dropdowns[category].options = ['Select'] + extract_keywords(merged_df, column_name)
    dropdowns[category].observe(on_dropdown_change, names='value')

initial_filters = {category: 'Select' for category in search_categories}
display_names(initial_filters)